# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Import" data-toc-modified-id="Data-Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Import</a></div><div class="lev2 toc-item"><a href="#Initial-Setup" data-toc-modified-id="Initial-Setup-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Initial Setup</a></div><div class="lev2 toc-item"><a href="#Unique-Players" data-toc-modified-id="Unique-Players-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Unique Players</a></div><div class="lev3 toc-item"><a href="#Finding-Unique-Players" data-toc-modified-id="Finding-Unique-Players-121"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Finding Unique Players</a></div><div class="lev3 toc-item"><a href="#Making-room-for-data." data-toc-modified-id="Making-room-for-data.-122"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Making room for data.</a></div><div class="lev2 toc-item"><a href="#Examination-of-Data" data-toc-modified-id="Examination-of-Data-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Examination of Data</a></div><div class="lev1 toc-item"><a href="#Creating-Extra-Stats" data-toc-modified-id="Creating-Extra-Stats-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Creating Extra Stats</a></div><div class="lev2 toc-item"><a href="#Definitions-Current-&amp;-New" data-toc-modified-id="Definitions-Current-&amp;-New-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Definitions Current &amp; New</a></div>

# Data Import

## Initial Setup

Importing necessary modules. Pandas for data manipulation. NumPy for array options. PPrint for nice displays of output when wanted. OS for input/output operations in Windows. RE for more than basic string filtration. 

In [21]:
import pandas as pd
import numpy as np
import pprint as pp
import os
import re
import csv

In [2]:
print(os.getcwd(), os.listdir())

C:\Users\Suren\Documents\TTU\Drive\Portal\Jupyter\Projects\Tennis_Analysis ['.ipynb_checkpoints', 'atp-tour-20002016.zip', 'Data.csv', 'desktop.ini', 'flagged_names.txt', 'Individual_Players', 'Metadata.txt', 'Suren_Test', 'Tennis_Analysis.ipynb']


In [3]:
raw_df = pd.read_csv('Data.csv', encoding='latin1')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46652 entries, 0 to 46651
Data columns (total 54 columns):
ATP           46652 non-null int64
Location      46652 non-null object
Tournament    46652 non-null object
Date          46652 non-null object
Series        46652 non-null object
Court         46652 non-null object
Surface       46652 non-null object
Round         46652 non-null object
Best of       46652 non-null int64
Winner        46652 non-null object
Loser         46652 non-null object
WRank         46637 non-null object
LRank         46581 non-null object
W1            46423 non-null float64
L1            46423 non-null float64
W2            45962 non-null float64
L2            45963 non-null float64
W3            21365 non-null float64
L3            21365 non-null float64
W4            4226 non-null float64
L4            4226 non-null float64
W5            1617 non-null float64
L5            1617 non-null float64
Wsets         46462 non-null float64
Lsets         46462 no

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dropping the labels that did not interest me.

In [4]:
dr_labels = [
    'B365W','B365L','B&WW','B&WL','CBW','CBL','EXW','EXL','LBW','LBL','GBW','GBL','IWW',
    'IWL','PSW','PSL','SBW','SBL','SJW','SJL','UBW','UBL', 'MaxW','MaxL','AvgW','AvgL'
]

raw_df = raw_df.drop(dr_labels, axis=1)

## Unique Players

### Finding Unique Players

In [5]:
win_arr = raw_df.Winner.unique()
los_arr = raw_df.Loser.unique()
print('Total unique number of Winners: ' + str(len(win_arr)))
print('Total unique number of Losers: ' + str(len(los_arr)))

unique_arr = np.concatenate((win_arr, los_arr))
print('Total unique number in both Winners and Losers: ' + str(len(unique_arr)))
print('Total unique players: ' + str(len(set(unique_arr))))

Total unique number of Winners: 879
Total unique number of Losers: 1383
Total unique number in both Winners and Losers: 2262
Total unique players: 1387


Found a comprehensive way to get the same result.

In [7]:
unique_set = set(pd.unique(raw_df[['Winner', 'Loser']].values.ravel()))
print('Total unique players: ' + str(len(unique_set)))
#pp.pprint(unique_set)

Total unique players: 1387


I noticed that there were some duplicates with spaces at the end. I'm going to take out any trailing spaces and see how much it reduces the unique set of players.

In [8]:
raw_df.Winner = raw_df.Winner.str.strip()
raw_df.Loser = raw_df.Loser.str.strip()

unique_set = set(pd.unique(raw_df[['Winner', 'Loser']].values.ravel()))
print('Total unique players: ' + str(len(unique_set)))
#pp.pprint(unique_set)

Total unique players: 1322


This reduced the number from 1387 to 1322. After scrolling through I noticed names like *Munoz De La Nava D.* had slightly different ways it was inputed in. Lowercase vs uppercase and hyphen in name led to four duplicates. One way I can think of to clean this is to turn all the letters in a name to lowercase for a base template, capitalize the first letter of every word in the full name,  and replace any hyphens with spaces.

In [9]:
raw_df.Winner = raw_df.Winner.str.lower().str.title().str.replace('-', ' ')
raw_df.Loser = raw_df.Loser.str.lower().str.title().str.replace('-', ' ')

unique_set = set(pd.unique(raw_df[['Winner', 'Loser']].values.ravel()))
print('Total unique players: ' + str(len(unique_set)))
#pp.pprint(unique_set)

Total unique players: 1308


Reduced to 1308 from 1322. Scrolling through again, I noticed *Kohlschreiber P.* was in there twice due to an extra period at the end. I can fix this too, but I want to see if there are any other duplicates that shouldn't be there.

To do this, I'll search the pandas series for partial matches for every name and append them to a list that will give me a chance to judge why they were partial duplicates. I expect many are just people with the same last names. But the rest I should be able to make judgment call and try to clean.

In [10]:
unique_df = pd.DataFrame(pd.unique(raw_df[['Winner', 'Loser']].values.ravel()))
uni_dupl_list = []
checked_names = []

for name in unique_df[0]:
    if name not in checked_names:
        temp = unique_df[unique_df[0].str.contains(name[0:4])][0]
        if len(temp) > 1:
            uni_dupl_list.append(list(temp))
            for temp_name in temp:
                checked_names.append(temp_name)
        
#pp.pprint(uni_dupl_list)

The names I've visually flagged: (I saved a text file with this list of lists.)

In [11]:
flagged_names = open('flagged_names.txt')
for line in flagged_names:
    print(line)

['Gambill J.M.', 'Gambill J. M.']

['Hantschk M.', 'Hantschek M.']

['Marin J.A.', 'Marin L.', 'Marin J.A']

['Schuettler R.', 'Schukin Y.', 'Schuettler P.', 'Schuttler P.']

['Sanchez J.', 'Sanchez D.', 'Sanchez De Luna J.A.', 'Sanchez De Luna J.', 'Sanchez M.']

['Rodriguez M.', 'Rodriguez B.', 'Rodrigues M.']

['Chela J.I.', 'Chela J.']

['Kim K.', 'Kim A.', 'Kim K']

['Kohlmann M.', 'Kohlschreiber P.', 'Kohlschreiber P..']

['Mathieu P.H.', 'Mathieu P.']

['Jones A.', 'Jones G.D.', 'Jones G.']

['Bogomolov Jr.A.', 'Bogomolov Jr. A.', 'Bogomolov A.']

['Wang Y.', 'Wang Y.T.', 'Wang Y. Jr', 'Wang Y.Jr.', 'Wang J.', 'Wang C.']

['Kunitcin I.', 'Kunitsyn I.']

['Gallardo Valles M.', 'Gallardo M.']

['Van Scheppingen D.', 'Scherrer J.C.', 'Scherrer J.', 'De Schepper K.']

['Lu Y.H.', 'Lu Y.']

['Qureshi A.', 'Qureshi A.UH.']

['Querry S.', 'Querrey S.']

['Haider Mauer A.', 'Haider Maurer A.']

['Jun W.', 'Jun W.S.']

['Stebe C.M.', 'Stebe C M.

['Viola Mat.', 'Viola M.']

['Zayed M. S.

Of the list I compiled, the easiest and high probabilty of input error are the ones having to do with spaces and periods. The rest are too ambigious for me too alter. But I'll keep this list as a caveat to note when presenting any of my analysis.

In the meantime, I'll clean up the data some more: 
- Strip extra periods at the end. 
- Add periods at the very end if not there.
- Add periods in between initials.
- Removing spaces for names with multiple initials.

Defining some functions that can be used later on.

In [42]:
def cln_low_upp_no_hyp(name):
    return name.lower().title().replace('-', ' ')

def cln_str_ext_per(name):
    while name[-2] == '.' and name[-1] == '.':
        name = name[:-1]
        
    return name

def cln_add_per(name):
    if name[-1] != '.':
        return (name + '.')
    else:
        return name
    
def cln_add_per_in(name):
    patterns = re.findall(r'[A-Z] ', name)
    if len(patterns) > 0:
        for pat in patterns:
            name = name.replace(pat, (pat[0] + '.'))
        
    return name

def cln_total(name):
    name = cln_add_per_in(cln_add_per(cln_str_ext_per(cln_low_upp_no_hyp(name)))).replace('. ', '.')
    return name


In [13]:
raw_df.Winner = raw_df.Winner.apply(cln_total)
raw_df.Loser = raw_df.Loser.apply(cln_total)

unique_set = set(pd.unique(raw_df[['Winner', 'Loser']].values.ravel()))
print('Total unique players: ' + str(len(unique_set)))
#pp.pprint(unique_set)

Total unique players: 1297


Final reduction to 1297. This is the most I'm comfortable cleaning without taking the chance of combining truly unique players.

In [26]:
pd.Series(list(unique_set)).to_csv('unique_set.csv')

### Making room for data.

A folder for each player to store data related to them. Information, pictures, etc.

Creating some translation functions when going from folder names to the initial data names.

In [27]:
def to_fol(name):
    return name[:-1].replace(' ', '_').replace('.', '-')

def from_fol(name):
    return (name.replace('-', '.').replace('_', ' ') + '.')


In [28]:
if 'Individual_Players' not in os.listdir():
    for player in unique_set:
        os.makedirs('Individual_Players/{}'.format(to_fol(player)))

## Examination of Data

In [29]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46652 entries, 0 to 46651
Data columns (total 28 columns):
ATP           46652 non-null int64
Location      46652 non-null object
Tournament    46652 non-null object
Date          46652 non-null object
Series        46652 non-null object
Court         46652 non-null object
Surface       46652 non-null object
Round         46652 non-null object
Best of       46652 non-null int64
Winner        46652 non-null object
Loser         46652 non-null object
WRank         46637 non-null object
LRank         46581 non-null object
W1            46423 non-null float64
L1            46423 non-null float64
W2            45962 non-null float64
L2            45963 non-null float64
W3            21365 non-null float64
L3            21365 non-null float64
W4            4226 non-null float64
L4            4226 non-null float64
W5            1617 non-null float64
L5            1617 non-null float64
Wsets         46462 non-null float64
Lsets         46462 no

There are 46,652 rows of data. The columns that have less than this need to be examined to figure out how to handle the null or empty data.

In [34]:
pd.set_option('display.max_columns', 28)
raw_df[raw_df.Wsets.isnull()]

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,WPts,LPts
9479,19,Indian Wells,Pacific Life Open,13/03/2003,Masters,Outdoor,Hard,2nd Round,3,Ginepri R.,Novak J.,80,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
9584,20,Miami,NASDAQ-100 Open,26/03/2003,Masters,Outdoor,Hard,4th Round,3,Srichaphan P.,Rios M.,13,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
10678,43,Kitzbuhel,Generali Open,24/07/2003,International Gold,Outdoor,Clay,3rd Round,3,Zabaleta M.,El Aynaoui Y.,30,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11229,55,Palermo,Campionati Internazionali Di Sicilia,25/09/2003,International,Outdoor,Clay,2nd Round,3,Squillari F.,Volandri F.,103,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11430,61,Vienna,CA Tennis Trophy,9/10/2003,International Gold,Indoor,Hard,2nd Round,3,Mirnyi M.,Calleri A.,28,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11516,63,Basel,Swiss Indoors,26/10/2003,International,Indoor,Carpet,The Final,5,Coria G.,Nalbandian D.,5,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11615,66,Paris,BNP Paribas,30/10/2003,Masters,Indoor,Carpet,3rd Round,3,Pavel A.,Coria G.,191,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11753,4,Auckland,Heineken Open,14/01/2004,International,Outdoor,Hard,2nd Round,3,Hrbaty D.,Coria G.,54,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN
11818,6,Melbourne,Australian Open,19/01/2004,Grand Slam,Outdoor,Hard,1st Round,5,Blanco G.,El Aynaoui Y.,99,18,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Retired,NaN,NaN
12481,20,Estoril,Estoril Open,16/04/2004,International Series,Outdoor,Clay,Quarterfinals,3,Labadze I.,Nadal R.,52,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Walkover,NaN,NaN


After reviewing the data a column at a time, the NaNs are acceptable for now.

Now to see if there are any typos that can be found in the other columns.

In [35]:
for column in raw_df.columns:
    print((column + ': '), pd.unique(raw_df[column]))

Index(['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'W1', 'L1',
       'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 'Wsets', 'Lsets',
       'Comment', 'WPts', 'LPts'],
      dtype='object')
ATP:  [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50
 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69]
Location:  ['Adelaide' 'Chennai' 'Doha' 'Auckland' 'Sydney' 'Melbourne' 'Dubai'
 'Marseille' 'San Jose' 'Memphis' 'Rotterdam' 'London' 'Mexico City'
 'Copenhagen' 'Delray Beach' 'Santiago' 'Bogota' 'Scottsdale'
 'Indian Wells' 'Miami' 'Atlanta' 'Casablanca' 'Estoril' 'Monte Carlo'
 'Barcelona' 'Mallorca' 'Munich' 'Orlando' 'Rome' 'Hamburg' 'St. Polten'
 'Paris' 'Halle' 'Queens Club' 'Nottingham' "'s-Hertogenbosch" 'Bastad'
 'Gstaad' 'Newport' 'Amsterdam' 'Stuttgart' 'Umag' 'Kitzbuhel'
 'Los Ang

Noticed:
- Location
    - Typo: "'s-Hertogenbosch"
    - Typo: Trailing Spaces
- Best of
    - Has numbers outside of 3 & 5
- WRank & LRank
    - Some of the data is in string format.
    - Possibly converting the floats to ints.
- Comment
    - 'Compleed'
    - 'Retied' & 'retired'
    - 'Walover'

Location

In [54]:
#pd.unique(raw_df.Location)

def cln_location(location):
    if location[0] == "'":
        location = location[1:]
        
    return location.strip()

raw_df.Location = raw_df.Location.apply(cln_location)

Best of

In [58]:
raw_df[(raw_df['Best of'] != 3) & (raw_df['Best of'] != 5)]

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,WPts,LPts
28885,8,Santiago,Movistar Open,1/02/2010,ATP250,Outdoor,Clay,3rd Round,-1,Granollers M.,Vassallo Arguello M.,112,99,4.0,6.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,Retired,483.0,535.0
28886,8,Santiago,Movistar Open,1/02/2010,ATP250,Outdoor,Clay,2nd Round,0,Luczak P.,Starace P.,73,64,7.0,6.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,631.0,707.0
28887,8,Santiago,Movistar Open,1/02/2010,ATP250,Outdoor,Clay,1st Round,1,Martin A.,Daniel M.,129,89,6.0,7.0,7.0,5.0,6.0,4.0,NaN,NaN,NaN,NaN,2.0,1.0,Completed,429.0,565.0
28888,8,Santiago,Movistar Open,1/02/2010,ATP250,Outdoor,Clay,0th Round,2,Chela J.I.,Aguilar J.,77,223,7.0,5.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,Completed,621.0,208.0


In [64]:
raw_df.loc[28885:28888, 'Best of'] = 3
print(raw_df.loc[28885:28888, 'Best of'])

raw_df[(raw_df['Best of'] != 3) & (raw_df['Best of'] != 5)]

28885    3
28886    3
28887    3
28888    3
Name: Best of, dtype: int64


,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,WPts,LPts


WRank & LRank

In [158]:
# raw_df.WRank.apply(lambda x: int(np.round(float(x))))
# I tried the simple conversion initially, but there was a non-number in there.

#print(raw_df[raw_df.WRank == 'NR'])
#raw_df.loc[744, 'WRank'] = np.NaN
#print(raw_df.loc[744])

# Found an easier way, but I had to replace the NaNs to zeros, 
# which are still readable considering there is no ranking of 0.
raw_df.WRank = pd.to_numeric(raw_df.WRank, errors='coerce', downcast='integer').replace(np.NaN, 0)
raw_df.LRank = pd.to_numeric(raw_df.LRank, errors='coerce', downcast='integer').replace(np.NaN, 0)

#raw_df.LRank

Comment
    - 'Compleed'
    - 'Retied' & 'retired'
    - 'Walover'

In [143]:
raw_df[raw_df.Comment == 'Compleed'] = 'Completed'
raw_df[(raw_df.Comment == 'Retied') | (raw_df.Comment == 'retired')] = 'Retired'
raw_df[raw_df.Comment == 'Walover'] = 'Walkover'

# Creating Extra Stats

## Definitions Current & New

In [144]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46652 entries, 0 to 46651
Data columns (total 28 columns):
ATP           46652 non-null object
Location      46652 non-null object
Tournament    46652 non-null object
Date          46652 non-null object
Series        46652 non-null object
Court         46652 non-null object
Surface       46652 non-null object
Round         46652 non-null object
Best of       46652 non-null object
Winner        46652 non-null object
Loser         46652 non-null object
WRank         46652 non-null object
LRank         46652 non-null object
W1            46424 non-null object
L1            46424 non-null object
W2            45965 non-null object
L2            45966 non-null object
W3            21398 non-null object
L3            21398 non-null object
W4            4276 non-null object
L4            4276 non-null object
W5            1667 non-null object
L5            1667 non-null object
Wsets         46463 non-null object
Lsets         46463 non-null ob

I always wanted to see if what percentage of players win a match by truly winning more games. Creating two new columns of Winner Total Games and Loser Total Games, should be useful in seeing the overall trend.

In [145]:
raw_df['WT'] = raw_df[['W1', 'W2', 'W3', 'W4', 'W5']].sum(axis=1)
raw_df['LT'] = raw_df[['L1', 'L2', 'L3', 'L4', 'L5']].sum(axis=1)

In [148]:
pd.set_option('display.max_columns', 30)
#raw_df[(raw_df.LT > raw_df.WT) & (raw_df['Best of'] == 5)]

Having the difference of the winner and loser ranks is a good metric to have.

In [185]:
raw_df['WL_RDiff'] = raw_df.WRank - raw_df.LRank

print(raw_df[raw_df.WL_RDiff == raw_df.WL_RDiff.min()][['Tournament', 'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'WL_RDiff']])
print(raw_df[raw_df.WL_RDiff == raw_df.WL_RDiff.max()][['Tournament', 'Round', 'Winner', 'Loser', 'WRank', 'LRank', 'WL_RDiff']])

                   Tournament      Round     Winner          Loser  WRank  \
33964  Brisbane International  The Final  Murray A.  Dolgopolov O.    4.0   

        LRank  WL_RDiff  
33964  7380.0   -7376.0  
                                 Tournament      Round     Winner     Loser  \
19944  Campionati Internazionali Di Sicilia  1st Round  Cermak F.  Pavel A.   

        WRank  LRank  WL_RDiff  
19944  1554.0   90.0    1464.0  


That should be good enough for explicit stats. All others can be made implicitly when needed.

Now to save the data that has been cleaned and modified.

In [190]:
raw_df.to_excel('Tennis_Data.xlsx')